# 3D vortex flow
We use the flowfield as defined in the paper: "Aggregation of slightly buoyant microplastics in 3D vortex flows"  
by I. Rypina, L. Pratt and M. Dotzel in Nonlinear processes in gheophsics, 25-33, 3(1) (2024)  
We test both the symmetric and assymetric 3D flow. 


In [ ]:
# import needed packages
import sys
sys.path.append('../')
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from datetime import datetime, timedelta 
from parcels import FieldSet, ParticleSet, ParticleFile
from parcels import AdvectionRK4
from functions.kernels import InertialParticle2D, InitializeParticles3D, MRAdvectionRK4_3D, MRSMAdvectionRK4_3D


plt.style.use('../python_style_Meike.mplstyle')

In [7]:
# print python version
%load_ext autoreload
%autoreload 2
%load_ext version_information
%version_information numpy, matplotlib, xarray, parcels

Software versions
Python 3.12.2 64bit [GCC 12.3.0]
IPython 8.22.2
OS Linux 4.18.0 553.22.1.el8_10.x86_64 x86_64 with glibc2.28
numpy 1.26.4
matplotlib 3.8.3
xarray 2024.2.0
parcels 3.0.2
Thu Oct 31 16:16:35 2024 CET

In [1]:
def cylinder_flow3D(x, y, z, a, b, c, r0 ,y0, beta, gamma, epsilon):
    Z, Y, X = np.meshgrid(z, y, x,indexing='ij')
    R = np.sqrt((X**2+Y**2))
    ux=-b*X*(1-2*Z)*(r0-R)/3.0-a*Y*(c+Z**2)+epsilon*(Y*(Y-y0+gamma)-(r0**2-R**2)/2.)*(1-beta*Z)
    uy=-b*Y*(1-2*Z)*(r0-R)/3.0+a*X*(c+Z**2)-epsilon*X*(Y-y0+gamma)*(1-beta*Z)
    uz=b*Z*(1-Z)*(2*r0-3*R)/3.0
    return X, Y, Z, ux, uy, uz

## Symmetric flow


In [ ]:
# settings for symmetric flow:
x=np.linspace(-0.5,0.5,101)
y=np.linspace(-0.5,0.5,101)
z=np.linspace(0,1,101)


a=0.62
b=7.5
c=0.69
r0=0.5
y0=0
beta=0
gamma=0
epsilon=0

X, Y, Z, ux, uy, uz = cylinder_flow3D(x,y,z,a,b,c,r0,y0,beta,gamma,epsilon) 

#save field in dictionary
data_symmetric_flow = {
    "U": ux,
    "V": uy,
    "W": uz,
    }

dims_symmetric_flow = {
    "lon": X,
    "lat": Y,
    "depth": Z
    }

## parcels simulation

In [ ]:
#fieldset
# set fieldset
fieldset = FieldSet.from_data(
    data = data_symmetric_flow,
    dimensions = dims_symmetric_flow,
    mesh="flat",
    allow_time_extrapolation=True
)
# add rotation here 0 as we have flat mesh
fieldset.add_constant('Omega_earth', 0) 
fieldset.add_constant('g', 10) 

# stepsize for finite differences calculation
Delta_x = fieldset.U.grid.lon[0,1]-fieldset.U.grid.lon[0,0]
Delta_y = fieldset.U.grid.lat[1,0]-fieldset.U.grid.lat[0,0]
delta_x = 0.5 * Delta_x
delta_y = 0.5 * Delta_y
fieldset.add_constant('delta_x', delta_x)
fieldset.add_constant('delta_y', delta_y)
print(f'(delta x, delta y) = ({delta_x}, {delta_y})')


In [ ]:
 #particle settings
B = 0.97
tau = 0.1